CELL 1: Setup & Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers torch safetensors

import torch
import json
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from safetensors.torch import load_file

drivepath = "/content/drive/MyDrive/intent_project"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Device: {device}")


Mounted at /content/drive
Device: cpu


CELL 2: Define CRF Model Class

In [3]:
import torch
import torch.nn as nn
from transformers import AutoModel

!pip install -q pytorch-crf
from torchcrf import CRF

class XLMRobertaWithCRF(nn.Module):
    def __init__(self, model_name, num_labels, id2label, label2id):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.transformer.config.hidden_size, num_labels)
        self.crf = CRF(num_labels, batch_first=True)
        self.id2label = id2label
        self.label2id = label2id
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            labels = torch.where(labels == -100, torch.zeros_like(labels), labels)
            loss = -self.crf(logits, labels, mask=attention_mask.bool(), reduction='mean')
            return {'loss': loss, 'logits': logits}
        else:
            predictions = self.crf.decode(logits, mask=attention_mask.bool())
            return {'logits': logits, 'predictions': predictions}

print("✓ CRF model class defined")

✓ CRF model class defined



CELL 3: Load CRF Model

In [4]:
print("=" * 70)
print("LOADING CRF MODEL")
print("=" * 70)

with open(f"{drivepath}/slot_filling_model_crf/final_model/id2label.json", 'r') as f:
    id2label_crf = json.load(f)
with open(f"{drivepath}/slot_filling_model_crf/final_model/label2id.json", 'r') as f:
    label2id_crf = json.load(f)

model_crf = XLMRobertaWithCRF(
    model_name="xlm-roberta-base",
    num_labels=len(label2id_crf),
    id2label=id2label_crf,
    label2id=label2id_crf
)

model_state_crf = load_file(f"{drivepath}/slot_filling_model_crf/final_model/model.safetensors")
model_crf.load_state_dict(model_state_crf)

tokenizer_crf = AutoTokenizer.from_pretrained(
    f"{drivepath}/slot_filling_model_crf/final_model"
)

model_crf.to(device)
model_crf.eval()

print(f"✓ CRF Model loaded")
print(f"  Labels: {len(label2id_crf)}")
print(f"  Device: {device}")


LOADING CRF MODEL


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

✓ CRF Model loaded
  Labels: 111
  Device: cpu


CELL 4: Load Non-CRF Model

In [5]:
print("=" * 70)
print("LOADING NON-CRF MODEL")
print("=" * 70)

with open(f"{drivepath}/ner_dataset_plain/id2label.json", 'r') as f:
    id2label_plain = json.load(f)
with open(f"{drivepath}/ner_dataset_plain/label2id.json", 'r') as f:
    label2id_plain = json.load(f)

model_plain = AutoModelForTokenClassification.from_pretrained(
    f"{drivepath}/slot_filling_model_plaintext/final_model"
)

tokenizer_plain = AutoTokenizer.from_pretrained(
    f"{drivepath}/slot_filling_model_plaintext/final_model"
)

model_plain.to(device)
model_plain.eval()

print(f"✓ Non-CRF Model loaded")
print(f"  Labels: {len(label2id_plain)}")
print(f"  Device: {device}")


LOADING NON-CRF MODEL
✓ Non-CRF Model loaded
  Labels: 111
  Device: cpu


CELL 5: CRF Inference Function

In [6]:
def extract_slots_crf(utterance, tokenizer, model, id2label, device):
    inputs = tokenizer(
        utterance.split(),
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask']
        )
        predictions = torch.tensor(outputs['predictions'][0])

    batch_encoding = tokenizer(
        utterance.split(),
        is_split_into_words=True,
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    word_ids = batch_encoding.word_ids(batch_index=0)
    words = utterance.split()

    word_predictions = []
    for word_idx in range(len(words)):
        word_preds = []
        for token_idx, wid in enumerate(word_ids):
            if wid == word_idx:
                word_preds.append(predictions[token_idx])
        if word_preds:
            word_predictions.append(word_preds[0])

    slots = []
    current_slot_type = None
    current_slot_words = []

    for word, pred_id in zip(words, word_predictions):
        label = id2label[str(int(pred_id))]

        if label == 'O':
            if current_slot_type:
                slots.append({
                    'type': current_slot_type,
                    'value': ' '.join(current_slot_words)
                })
            current_slot_type = None
            current_slot_words = []

        elif label.startswith('B-'):
            if current_slot_type:
                slots.append({
                    'type': current_slot_type,
                    'value': ' '.join(current_slot_words)
                })
            current_slot_type = label[2:]
            current_slot_words = [word]

        elif label.startswith('I-'):
            slot_type = label[2:]
            if slot_type == current_slot_type:
                current_slot_words.append(word)
            else:
                if current_slot_type:
                    slots.append({
                        'type': current_slot_type,
                        'value': ' '.join(current_slot_words)
                    })
                current_slot_type = slot_type
                current_slot_words = [word]

    if current_slot_type:
        slots.append({
            'type': current_slot_type,
            'value': ' '.join(current_slot_words)
        })

    return slots

print("✓ CRF inference function ready")


✓ CRF inference function ready



CELL 6: Plain Inference Function

In [7]:
def extract_slots_plain(utterance, tokenizer, model, id2label, device):
    inputs = tokenizer(
        utterance.split(),
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).squeeze().cpu().numpy()

    batch_encoding = tokenizer(
        utterance.split(),
        is_split_into_words=True,
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    word_ids = batch_encoding.word_ids(batch_index=0)
    words = utterance.split()

    word_predictions = []
    for word_idx in range(len(words)):
        word_preds = []
        for token_idx, wid in enumerate(word_ids):
            if wid == word_idx:
                word_preds.append(predictions[token_idx])
        if word_preds:
            word_predictions.append(word_preds[0])

    slots = []
    current_slot_type = None
    current_slot_words = []

    for word, pred_id in zip(words, word_predictions):
        label = id2label[str(int(pred_id))]

        if label == 'O':
            if current_slot_type:
                slots.append({
                    'type': current_slot_type,
                    'value': ' '.join(current_slot_words)
                })
            current_slot_type = None
            current_slot_words = []

        elif label.startswith('B-'):
            if current_slot_type:
                slots.append({
                    'type': current_slot_type,
                    'value': ' '.join(current_slot_words)
                })
            current_slot_type = label[2:]
            current_slot_words = [word]

        elif label.startswith('I-'):
            slot_type = label[2:]
            if slot_type == current_slot_type:
                current_slot_words.append(word)
            else:
                if current_slot_type:
                    slots.append({
                        'type': current_slot_type,
                        'value': ' '.join(current_slot_words)
                    })
                current_slot_type = slot_type
                current_slot_words = [word]

    if current_slot_type:
        slots.append({
            'type': current_slot_type,
            'value': ' '.join(current_slot_words)
        })

    return slots

print("✓ Plain inference function ready")


✓ Plain inference function ready


CELL 7: Comparison

In [8]:
print("=" * 90)
print("COMPARING CRF vs NON-CRF MODELS")
print("=" * 90)

test_utterances = [
    "wake me up at 6 AM tomorrow",
    "order a cappuccino and a croissant",
    "book a table at an Italian restaurant at 7pm",
    "play some jazz music by Miles Davis",
    "remind me to call John next Friday",
    "what's the weather in Paris this weekend",
    "set alarm for 8 AM on Monday",
]

matches = 0
for i, utterance in enumerate(test_utterances, 1):
    print(f"\n{'=' * 90}")
    print(f"Example {i}: {utterance}")
    print('=' * 90)

    slots_crf = extract_slots_crf(utterance, tokenizer_crf, model_crf, id2label_crf, device)
    slots_plain = extract_slots_plain(utterance, tokenizer_plain, model_plain, id2label_plain, device)

    print(f"\n🔷 CRF Model ({len(slots_crf)} slots):")
    if slots_crf:
        for slot in slots_crf:
            print(f"    [{slot['type']}]: {slot['value']}")
    else:
        print(f"    No slots detected")

    print(f"\n🔶 Non-CRF Model ({len(slots_plain)} slots):")
    if slots_plain:
        for slot in slots_plain:
            print(f"    [{slot['type']}]: {slot['value']}")
    else:
        print(f"    No slots detected")

    if slots_crf == slots_plain:
        print(f"\n✅ SAME OUTPUT")
        matches += 1
    else:
        print(f"\n⚠️ DIFFERENT OUTPUT")

print(f"\n{'=' * 90}")
print(f"SUMMARY: {matches}/{len(test_utterances)} examples match")
print('=' * 90)


COMPARING CRF vs NON-CRF MODELS

Example 1: wake me up at 6 AM tomorrow

🔷 CRF Model (2 slots):
    [time]: 6 AM
    [date]: tomorrow

🔶 Non-CRF Model (2 slots):
    [time]: 6 AM
    [date]: tomorrow

✅ SAME OUTPUT

Example 2: order a cappuccino and a croissant

🔷 CRF Model (2 slots):
    [food_type]: cappuccino
    [food_type]: croissant

🔶 Non-CRF Model (2 slots):
    [coffee_type]: cappuccino
    [drink_type]: croissant

⚠️ DIFFERENT OUTPUT

Example 3: book a table at an Italian restaurant at 7pm

🔷 CRF Model (3 slots):
    [food_type]: Italian
    [business_type]: restaurant
    [time]: 7pm

🔶 Non-CRF Model (3 slots):
    [food_type]: Italian
    [business_type]: restaurant
    [time]: 7pm

✅ SAME OUTPUT

Example 4: play some jazz music by Miles Davis

🔷 CRF Model (2 slots):
    [music_genre]: jazz
    [artist_name]: Miles Davis

🔶 Non-CRF Model (2 slots):
    [music_genre]: jazz
    [artist_name]: Miles Davis

✅ SAME OUTPUT

Example 5: remind me to call John next Friday

🔷 CRF Mod